## Estructura de la Aplicación:

- **Verificación del backup:**
    - Detectar la presencia del archivo DBSIF.FBK.
    - Verificar la fecha y hora de creación/modificación.
    - Obtener el tamaño del archivo.

- **Notificación por Email y/o WhatsApp:**
    - Si el backup es exitoso (el archivo cumple con los criterios), se envía una notificación.
    - En caso de fallo (si no se genera el archivo o tiene un tamaño inesperado), enviar una alerta.

**Requisitos:**

- Bibliotecas necesarias:
    - `os` para verificar el archivo.
    - `smtplib` para enviar emails.
    - API de WhatsApp (podemos usar Twilio o alguna otra plataforma que permita la integración).
    - `schedule` o utilizar una tarea programada en Windows para ejecutar la verificación.

## Verificación del archivo DBSIF.FBK:


In [1]:
# Importamos librerías

import os
import time
import smtplib
import ssl
import configparser

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


## Configuraciones
- email envío
- contraseña con variables de entorno
- Ruta de la DB
- email destino

Para configurar los parámetros de tu aplicación en un archivo `config.ini`, puedes utilizar el módulo `configparser de Python`, que facilita la lectura y escritura de archivos de configuración.

El archivo `config.ini` sigue una estructura en la que los parámetros se agrupan en secciones, y cada sección contiene claves y valores. A continuación, te muestro un ejemplo básico y cómo utilizarlo en tu código Python:

1. **Ejemplo de archivo config.ini:**

```python
[Database]
host = localhost
port = 5432
user = admin
password = secret

[Logging]
log_level = INFO
log_file = /var/log/myapp.log

[API]
endpoint = https://api.example.com
api_key = your-api-key
timeout = 30
```

- [Database]: Parámetros relacionados con la base de datos.
- [Logging]: Configuraciones de registro/logging.
- [API]: Configuración para acceso a una API.

2. **Cómo leer el archivo config.ini desde Python:**
Primero, necesitas importar el módulo configparser y luego leer el archivo config.ini para acceder a los valores.

```Python

import configparser

# Crear un objeto ConfigParser
config = configparser.ConfigParser()

# Leer el archivo de configuración
config.read('config.ini')

# Acceder a los parámetros
db_host = config['Database']['host']
db_port = config['Database'].getint('port')  # convertimos a entero
db_user = config['Database']['user']
db_password = config['Database']['password']

log_level = config['Logging']['log_level']
log_file = config['Logging']['log_file']

api_endpoint = config['API']['endpoint']
api_key = config['API']['api_key']
timeout = config['API'].getint('timeout')  # convertimos a entero

# Mostrar algunos valores para verificar
print(f"Connecting to database on {db_host}:{db_port} as {db_user}")
print(f"Logging at {log_level} level to {log_file}")
print(f"API endpoint: {api_endpoint}, Timeout: {timeout} seconds") 
```


3. **Ventajas de usar config.ini:**

- *Separación de código y configuración:*
Los parámetros de configuración no están embebidos en el código, lo que facilita su modificación sin tocar la lógica de la aplicación.

- *Facilidad de uso:*
Es simple de leer y modificar tanto para humanos como para programas.
Flexibilidad: Puedes agregar nuevas secciones y parámetros sin cambiar el código de la aplicación.







In [2]:

# Crear un objeto ConfigParser
config = configparser.ConfigParser()

# Leer el archivo de configuración
config.read('config.ini')

# Acceder a los parámetros
backup_path = config['Backup']['path']
# Opción: Asegurar que las barras estén correctamente escapadas
backup_path = backup_path.replace("\\", "/")  # o usa "\\\\" para barras dobles

email_origin = config['Email']['email_origin']
email_destination = config['Email']['email_destination']
email_cc = config['Email']['email_cc'] if config['Email']['email_cc'] else None
email_cco = config['Email']['email_cco'] if config['Email']['email_cco'] else None

# Obtener la variable de entorno para la clave del correo
email_password = os.getenv(config['Email']['env_variable_key'])

# Mostrar algunos valores para verificar
print(f"Backup path: {backup_path}")
print(f"Email Origin: {email_origin}")
print(f"Email Destination: {email_destination}")
print(f"Email CC: {email_cc}")
print(f"Email CCO: {email_cco}")
print(f"Email Password (from env): {email_password if email_password else 'No environment variable set'}")



Backup path: D://Etsol//Clientes//Act Yapur//DBSIF.FBK
Email Origin: notificador.backup2024@gmail.com
Email Destination: csaluzzo@gmail.com
Email CC: None
Email CCO: None
Email Password (from env): gaav nwjq ihll pogd


In [3]:
def verificar_backup(ruta_backup):
    # Verificar si el archivo existe
    if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB
        tamano = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        # Obtener fecha de creación/modificación
        timestamp = os.path.getmtime(ruta_backup)
        fecha_creacion = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))

        return {"existe": True, "tamano_gb": tamano, "fecha_creacion": fecha_creacion}
    else:
        return {"existe": False}


Seguridad: Nunca debes exponer contraseñas directamente en el código, ya que esto pone en riesgo tus credenciales. Usa variables de entorno o servicios seguros para gestionar tus credenciales.

Acceso a Gmail: Gmail puede bloquear el acceso de aplicaciones no seguras. Es probable que necesites configurar Gmail para permitir aplicaciones menos seguras, o mejor aún, crear una contraseña de aplicaciones si tienes la verificación en dos pasos activada.


In [4]:


def enviar_email(asunto, cuerpo,  mail_o, mail_d, mail_cc, mail_cco, mail_pass):
    # "Backup Fallido", mensaje, mail_o, mail_d, mail_cc, mail_cco, mail_pass

    # Configura los detalles del correo
    """
    sender_email = "notificador.backup2024@gmail.com"  # Cambia esto por tu correo
    app_password = os.getenv("APP_PYTHON")  # Obtén la App Password desde la variable de entorno
    receiver_email = mail_d  # Cambia esto por el correo del receptor
    subject = asunto
    body = cuerpo"""

    # Crear el mensaje
    message = MIMEMultipart()
    message["From"] = mail_o
    message["To"] = mail_d
    message["Cc"] = mail_cc
    message["Cco"] = mail_cco
    message["Subject"] = asunto
    message.attach(MIMEText(cuerpo, "plain"))

    """message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))"""




    # Conexión al servidor SMTP de Gmail
    smtp_server = "smtp.gmail.com"
    port = 465  # Puerto SSL

    # Crear una conexión segura con SSL
    context = ssl.create_default_context()

    try:
        # Conectarse al servidor de correo
        with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
            """server.login(sender_email, app_password)  # Autenticarse con tu cuenta de Gmail
            server.sendmail(sender_email, mail_d, message.as_string())  # Enviar el correo  """

            server.login(mail_o, mail_pass)  # Autenticarse con tu cuenta de Gmail
            server.sendmail(mail_o, mail_d, message.as_string())  # Enviar el correo

        print("Correo enviado exitosamente")
    except Exception as e:
        print(f"Error: {e}")


Para mejorar la seguridad y funcionamiento:

1- **Uso de variables de entorno** para la contraseña: Puedes almacenar tu contraseña en una variable de entorno en lugar de escribirla directamente en el código. Aquí un ejemplo de cómo hacerlo:

Puedes establecer la variable de entorno `EMAIL_PASSWORD` en tu sistema de la siguiente manera:

- En Windows:
`setx EMAIL_PASSWORD "tu_contraseña"`

- En Linux/MacOS:
`export EMAIL_PASSWORD="tu_contraseña"`

2- **Habilitar aplicaciones menos seguras en Gmail**: Si no tienes activada la verificación en dos pasos, puedes permitir que aplicaciones menos seguras accedan a tu cuenta de Gmail:
- Ve a Permitir el acceso de aplicaciones menos seguras y actívalo.

3- **Uso de contraseñas de aplicación**: Si tienes activada la autenticación de dos pasos, lo ideal sería generar una contraseña de aplicación específica para tu script, en lugar de utilizar tu contraseña principal.

## Envío de notificaciones por WhatsApp: Usando la API de Twilio:

In [ ]:
# pip install twilio

from twilio.rest import Client

def enviar_whatsapp(mensaje):
    account_sid = 'tu_account_sid'
    auth_token = 'tu_auth_token'
    client = Client(account_sid, auth_token)

    message = client.messages.create(
        body=mensaje,
        from_='whatsapp:+14155238886',  # Número de Twilio
        to='whatsapp:+tu_numero'
    )
    print("WhatsApp enviado:", message.sid)

## Integración: Unir la verificación del backup con el envío de notificaciones:

In [6]:
def verificar_y_notificar(path, mail_o, mail_d, mail_cc, mail_cco, mail_pass):
    
    resultado = verificar_backup(path)

    
    if resultado['existe']:
        mensaje = f"Backup exitoso.\nTamaño: {resultado['tamano_gb']:.2f} GB\nFecha de creación: {resultado['fecha_creacion']}"
        enviar_email("Backup Exitoso", mensaje, mail_o, mail_d, mail_cc, mail_cco, mail_pass)
        # enviar_whatsapp(mensaje)
    else:
        mensaje = "ERROR: No se ha generado el archivo de backup."
        enviar_email("Backup Fallido", mensaje, mail_o, mail_d, mail_cc, mail_cco, mail_pass)
        # enviar_whatsapp(mensaje)
    print(mensaje)

verificar_y_notificar(backup_path, email_origin, email_destination, email_cc, email_cco, email_password)




""" ANTES DE CAMBIOS

def verificar_y_notificar():
    resultado = verificar_backup('D:\\Etsol\\Clientes\\Act Yapur\\DBSIF.FBK')

    if resultado['existe']:
        mensaje = f"Backup exitoso.\nTamaño: {resultado['tamano_gb']:.2f} GB\nFecha de creación: {resultado['fecha_creacion']}"
        enviar_email("Backup Exitoso", mensaje, "csaluzzo@gmail.com")
        # enviar_whatsapp(mensaje)
    else:
        mensaje = "ERROR: No se ha generado el archivo de backup."
        enviar_email("Backup Fallido", mensaje, "csaluzzo@gmail.com")
        # enviar_whatsapp(mensaje)
    print(mensaje)

verificar_y_notificar()
"""


Correo enviado exitosamente
ERROR: No se ha generado el archivo de backup.


' ANTES DE CAMBIOS\n\ndef verificar_y_notificar():\n    resultado = verificar_backup(\'D:\\Etsol\\Clientes\\Act Yapur\\DBSIF.FBK\')\n\n    if resultado[\'existe\']:\n        mensaje = f"Backup exitoso.\nTamaño: {resultado[\'tamano_gb\']:.2f} GB\nFecha de creación: {resultado[\'fecha_creacion\']}"\n        enviar_email("Backup Exitoso", mensaje, "csaluzzo@gmail.com")\n        # enviar_whatsapp(mensaje)\n    else:\n        mensaje = "ERROR: No se ha generado el archivo de backup."\n        enviar_email("Backup Fallido", mensaje, "csaluzzo@gmail.com")\n        # enviar_whatsapp(mensaje)\n    print(mensaje)\n\nverificar_y_notificar()\n'

In [29]:
def verificar_backup_2(ruta_backup):
    # Verificar si el archivo existe
    if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB
        tamano = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        # Obtener fecha de creación/modificación
        timestamp = os.path.getmtime(ruta_backup)
        fecha_creacion = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))

        return {"existe": True, "tamano_gb": tamano, "fecha_creacion": fecha_creacion}
    else:
        return {"existe": False}



ruta_prueba = "D:\\Etsol\\Clientes\\Act Yapur\\DBSIF.FBK"
print(backup_path)
prueba = verificar_backup_2(backup_path)
print(prueba)



D:/Etsol/Clientes/Act Yapur/DBSIF.FBK
{'existe': True, 'tamano_gb': 6.917958736419678, 'fecha_creacion': '2023-08-31 23:06:29'}


# Pruebas de funcionamiento del envío de mail
-  con cuenta `@gmail.com`

In [9]:


# Configura los detalles del correo
sender_email = "notificador.backup2024@gmail.com"  # Cambia esto por tu correo
# app_password = "gaav nwjq ihll pogd"     # Coloca aquí tu App Password de Gmail
app_password = os.getenv("APP_PYTHON")  # Obtén la App Password desde la variable de entorno
receiver_email = "csaluzzo@gmail.com"  # Cambia esto por el correo del receptor
subject = "Prueba de envío de correo"
body = "Este es un correo de prueba enviado desde Python utilizando App Passwords."

# Crear el mensaje
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message.attach(MIMEText(body, "plain"))

# Conexión al servidor SMTP de Gmail
smtp_server = "smtp.gmail.com"
port = 465  # Puerto SSL

# Crear una conexión segura con SSL
context = ssl.create_default_context()

try:
    # Conectarse al servidor de correo
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, app_password)  # Autenticarse con tu cuenta de Gmail
        server.sendmail(sender_email, receiver_email, message.as_string())  # Enviar el correo
    print("Correo enviado exitosamente")
except Exception as e:
    print(f"Error: {e}")


Correo enviado exitosamente


In [8]:
"""

import os
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Configura los detalles del correo
sender_email = "notificador.backup2024@gmail.com"  # Cambia esto por tu correo
app_password = os.getenv("APP_PYTHON")  # Obtén la App Password desde la variable de entorno
print(app_password)
receiver_email = "csaluzzo@gmail.com"  # Cambia esto por el correo del receptor
subject = "Prueba de envío de correo"
body = "Este es un correo de prueba enviado desde Python utilizando App Passwords y una variable de entorno."

# Crear el mensaje
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message.attach(MIMEText(body, "plain"))

# Conexión

"""

gaav nwjq ihll pogd
